From Dan Simon's "Optimal State Estimation", example 13.4: How to do parameter estimate using Kalman filter (EKF). And also Problem 13.23, which is to demonstrate that though the parameter is adjusted correctly, it may prevent the filter from converging to the true value. 

## Estimate System Parameters

Consider following state equation

$$\ddot{x}_1 + 2\zeta\omega_n\dot{x}_1+\omega_n^2 x_1 = \omega_n^2 w $$

where $\omega_n$ is the natural frequency and $\zeta$ is the damping ratio. Input $w$ is zero-mean noise. The state space representation can be written as

$$\begin{aligned}
\dot{x}_1 =& x_2  \\
\dot{x}_2 =& -\omega_n^2 x_1 - 2\zeta\omega_n x_2 + \omega_n^2 w
\end{aligned}
$$

In matrix form, it is

$$\begin{aligned}
\left[\begin{aligned} \dot{x}_1 \\ \dot{x}_2 \end{aligned}\right]
 =& \left[\begin{matrix} 
 0 & 1  \\ 
 -\omega_n^2 & - 2\zeta\omega_n 
 \end{matrix}\right] 
 \left[\begin{aligned} x_1 \\ x_2 \end{aligned}\right] + 
 \left[\begin{matrix} 
 0   \\ 
 \omega_n^2  
 \end{matrix}\right] w \\
 y = & \left[\begin{matrix} 
 1 & 0  \\ 
 0 & 1 
 \end{matrix}\right] 
 \left[\begin{aligned} x_1 \\ x_2 \end{aligned}\right]
\end{aligned}
$$

## Assumption

Assume $-2\zeta\omega_n$ is known, but $\zeta$ and $\omega$ are unknown. We want to estimate $-\omega_n^2$. Assume both $x_1$ and $x_2$ are available. define the known parameter as $b$, and $b=-\zeta\omega_n$. Define the parameter to be estimated as $x_3=-\omega_n^2$. The state space becomes

$$\begin{aligned}
\dot{x}_1 =& x_2  \\
\dot{x}_2 =& x_3 x_1 + b x_2  - x_3 w \\
\dot{x}_3 =& w_p
\end{aligned}
$$
Note that above state-space is a nonlinear system due to <span style='color:Blue'>$x_3 x_1$</span>. In order to apply Kalman filter. The system should be linearized and then use EKF. It's important to note that <span style='color:red'>$x_3$ is not measurable</span>. Therefore $y$ only includes $x_1$ and $x_2$. Otherwise, there is no point to *estimate* it. The linearized system is

$$\begin{aligned}
\left[\begin{aligned} \dot{x}_1 \\ \dot{x}_2 \\ \dot{x}_3 \end{aligned}\right]
 =& \left[\begin{matrix} 
 0 & 1 & 0 \\ 
 \hat{x}_3 & b & \hat{x}_1-w \\
 0 & 0 & 0
 \end{matrix}\right] 
 \left[\begin{aligned} x_1 \\ x_2 \\ x_3 \end{aligned}\right] + 
 \left[\begin{matrix} 
 0 & 0  \\ 
 -\hat{x}_3 & 0 \\
 0 & 1
 \end{matrix}\right] 
 \left[\begin{matrix} 
 w   \\ 
 w_p
 \end{matrix}\right]  \\
 y =& \left[\begin{matrix} 
 1 & 0 & 0\\ 
 0 & 1 & 0
 \end{matrix}\right] 
 \left[\begin{aligned} x_1 \\ x_2 \\ x_3 \end{aligned}\right]
\end{aligned}
$$

In [4]:
import numpy as np
import matplotlib.pyplot as plt
def Kalman_gain(P_priori, H, R):
    K = P_priori * H / (H * P_priori * H + R)
    return K

def Joseph_P_post(P_priori, H, R, K):
    P_posteriori = (1-K*H) * P_priori * (1-K*H) + K * R * K
    return P_posteriori

def non_Joseph_P_post(P_priori, H, R, K):
    P_posteriori = (1-K*H) * P_priori
    return P_posteriori

P_priori = 2.0
H = 0.8
R = 0.1
K = Kalman_gain(P_priori, H, R)
print('Kalman gain is: ', K)
print('Estimate (co)variance is: ', Joseph_P_post(P_priori, H, R, K))

Kalman gain is:  1.1594202898550723
Estimate (co)variance is:  0.14492753623188404
